# 3. Task的用法

In [1]:
using System.Threading;
using System.Threading.Tasks;

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
public static class TaskClass
{
    public static void TaskMethod(string taskname, int duration)
    {
        Console.WriteLine($"Task [{taskname}] is start... ...");
        Console.WriteLine($"Task [{taskname}] is running on a thread id: {Thread.CurrentThread.ManagedThreadId}, is thread pool thread: {Thread.CurrentThread.IsThreadPoolThread}");
        Thread.Sleep(duration);
        Console.WriteLine($"Task [{taskname}] is stop... ...");
    }

    public static void TaskReport(Task task)
    {
        Console.WriteLine($"任务完成，完成时候的状态为： IsCanceled={task.IsCanceled}  IsCompleted={task.IsCompleted}  IsFaulted={task.IsFaulted}");
    }
    
    public static int TaskMethodInt(string taskname, int duration)
    {
        Console.WriteLine($"Task [{taskname}] is start... ...");
        Console.WriteLine($"Task [{taskname}] is running on a thread id: {Thread.CurrentThread.ManagedThreadId}, is thread pool thread: {Thread.CurrentThread.IsThreadPoolThread}");
        Thread.Sleep(duration);
        Console.WriteLine($"Task [{taskname}] is stop... ...");

        return 1024;
    }
}

## 2.2 组合任务 ContinueWith

### 2.2.1 demo

In [3]:
//创建一个任务
Task<int> task = new Task<int>(() =>
{
    int sum = 0;
    Console.WriteLine("使用Task执行异步操作.");
    for (int i = 0; i < 100; i++)
    {
        sum += i;
    }
    return sum;
});

//启动任务，并安排到当前任务队列线程中执行任务(System.Threading.Tasks.TaskScheduler)
task.Start();
Console.WriteLine("主线程执行其他处理... ...");

//任务完成时执行处理
Task cwt = task.ContinueWith(t =>
{
    Console.WriteLine("任务完成后的执行结果：{0}", t.Result.ToString());
});
task.Wait();
cwt.Wait();

主线程执行其他处理... ...
使用Task执行异步操作.
任务完成后的执行结果：4950


### 2.2.2 任务的串行与并行

In [4]:
using System.Collections.Concurrent;

ConcurrentStack<int> stack = new ConcurrentStack<int>();

//t1先串行
var t1 = Task.Factory.StartNew(() =>
{
    stack.Push(1);
    stack.Push(2);
});

//t2,t3并行执行
var t2 = t1.ContinueWith(t =>
{
    int result;
    stack.TryPop(out result);
    Console.WriteLine("Task t2 result={0}, Thread id is {1}", result, Thread.CurrentThread.ManagedThreadId);
});

//t2,t3并行执行
var t3 = t1.ContinueWith(t =>
{
    int result;
    stack.TryPop(out result);
    Console.WriteLine("Task t3 result={0}, Thread id is {1}", result, Thread.CurrentThread.ManagedThreadId);
});

//等待t2和t3执行完
Task.WaitAll(t2, t3);

//t4串行执行
var t4 = Task.Factory.StartNew(() =>
{
    Console.WriteLine("当前集合元素个数：{0}, Thread id is {1}", stack.Count, Thread.CurrentThread.ManagedThreadId);
});
t4.Wait();

Task t3 result=2, Thread id is 30
Task t2 result=1, Thread id is 28
当前集合元素个数：0, Thread id is 5


### 2.2.3 子任务

In [5]:
Task<string[]> parent = new Task<string[]>(state =>
{
    Console.WriteLine(state);
    string[] result = new string[2];
    //创建并启动子任务
    new Task(() => { result[0] = "我是子任务1。"; }, TaskCreationOptions.AttachedToParent).Start();
    new Task(() => { result[1] = "我是子任务2。"; }, TaskCreationOptions.AttachedToParent).Start();
    
    return result;
}, "我是父任务，并在我的处理过程中创建多个子任务，所有子任务完成以后我才会结束执行。");

//任务处理完成后执行的操作
parent.ContinueWith(t =>
{
    Array.ForEach(t.Result, r => Console.WriteLine(r));
});

//启动父任务
parent.Start();

//等待任务结束Wait只能等待父线程结束，没办法等到父线程的ContinueWith结束
parent.Wait();

我是父任务，并在我的处理过程中创建多个子任务，所有子任务完成以后我才会结束执行。
我是子任务1。
我是子任务2。



(13,1): warning CS4014: Because this call is not awaited, execution of the current method continues before the call is completed. Consider applying the 'await' operator to the result of the call.



### 2.2.4 动态并行(TaskCreationOptions.AttachedToParent)
    
父任务等待所有子任务完成后，整个任务才算完成

In [6]:
Node root = GetNode();
DisplayTree(root);

public class Node
{
    public Node Left { get; set; }
    public Node Right { get; set; }
    public string Text { get; set; }
}

public static Node GetNode()
{
    Node root = new Node
    {
        Left = new Node {
            Left = new Node { Text = "L-L" },
            Right = new Node { Text = "L-R" },
            Text = "L"
        },
        Right = new Node {
            Left = new Node { Text = "R-L" },
            Right = new Node { Text = "R-R" },
            Text = "R"
        },
        Text = "Root"
    };

    return root;
}

public static void DisplayTree(Node root)
{
    var task = Task.Factory.StartNew(() => DisplayNode(root), CancellationToken.None, TaskCreationOptions.None, TaskScheduler.Default);
    task.Wait();
}

public static void DisplayNode(Node current)
{

    if (current.Left != null)
        Task.Factory.StartNew(() => DisplayNode(current.Left), CancellationToken.None, TaskCreationOptions.AttachedToParent, TaskScheduler.Default);
    if (current.Right != null)
        Task.Factory.StartNew(() => DisplayNode(current.Right), CancellationToken.None, TaskCreationOptions.AttachedToParent, TaskScheduler.Default);

    Console.WriteLine("当前节点的值为{0}; 处理的ThreadId={1}", current.Text, Thread.CurrentThread.ManagedThreadId);
}

当前节点的值为Root; 处理的ThreadId=5
当前节点的值为R; 处理的ThreadId=28
当前节点的值为L-L; 处理的ThreadId=32
当前节点的值为L; 处理的ThreadId=30
当前节点的值为R-R; 处理的ThreadId=31
当前节点的值为R-L; 处理的ThreadId=4
当前节点的值为L-R; 处理的ThreadId=33
